In [31]:
torch.backends.mps.is_available() 

True

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD, Adam
from pytorch_lightning.loggers import TensorBoardLogger, logger


import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import optuna
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


/Users/lsacy/opt/miniconda3/envs/clcnlu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [71]:
# load data
df = pd.read_csv('data/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [72]:
# convert categorical data 'smoker' and 'region' to numerical data
df['smoker'] = df['smoker'].map({'yes': 1, 'no': 0})
df['region'] = df['region'].map({'southwest': 0, 'southeast': 1, 'northwest': 2, 'northeast': 3})

# normalize data
df['age'] = (df['age'] - df['age'].mean()) / df['age'].std()
df['bmi'] = (df['bmi'] - df['bmi'].mean()) / df['bmi'].std()
df['children'] = (df['children'] - df['children'].mean()) / df['children'].std()
df['charges'] = (df['charges'] - df['charges'].mean()) / df['charges'].std()

# split data into train and validate and test
train_df = df.sample(frac=0.6, random_state=0)
val_df = df.drop(train_df.index).sample(frac=0.5, random_state=0)
test_df = df.drop(train_df.index).drop(val_df.index)

In [73]:
# create Dataset class for train_df and val_df

class trainDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.X = torch.tensor(df[['age', 'bmi', 'children', 'smoker', 'region']].values, dtype=torch.float32)
        self.y = torch.tensor(df['charges'].values, dtype=torch.float32)
        #self.y = self.y.view(self.y.shape[0], 1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class valDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.X = torch.tensor(df[['age', 'bmi', 'children', 'smoker', 'region']].values, dtype=torch.float32)
        self.y = torch.tensor(df['charges'].values, dtype=torch.float32)
        #self.y = self.y.view(self.y.shape[0], 1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# create train and val dataloaders
train_dataset = trainDataset(train_df)
val_dataset = valDataset(val_df)

In [75]:
train_dataset[0]

(tensor([ 0.9105, -0.0760, -0.0787,  0.0000,  0.0000]), tensor(-0.2928))

In [132]:
import optuna
import pytorch_lightning as pl
import torch

class RegressionModel(pl.LightningModule):
    def __init__(self, num_neurons, learning_rate, batch_size):
        super().__init__()
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.num_neurons = num_neurons              
        self.linear1 = torch.nn.Linear(5, num_neurons)
        self.linear2 = torch.nn.Linear(num_neurons, num_neurons)
        self.linear3 = torch.nn.Linear(num_neurons, 1)
        self.relu = torch.nn.ReLU()
        self.save_hyperparameters()

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.linear3(x)
        x = F.softmax(x, dim=1)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x).squeeze(dim=1)
        loss = torch.nn.functional.mse_loss(y_hat, y)
        self.log('train_loss', loss)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x).squeeze(dim=1)
        loss = torch.nn.functional.mse_loss(y_hat, y)
        self.log('val_loss', loss)
        return loss

    def validation_end(self, validation_step_outputs):
        avg_loss = torch.stack(validation_step_outputs)
        self.log('avg_val_loss', avg_loss)
        return avg_loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(train_dataset, batch_size=self.batch_size, num_workers=0)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(val_dataset, batch_size=self.batch_size, num_workers=0)




In [133]:
logger = TensorBoardLogger('lightning_logs', name='insurance')

model = RegressionModel(num_neurons=32, learning_rate=0.01, batch_size=32)
trainer = pl.Trainer(accelerator='gpu', devices=1, max_epochs=10, logger=logger)
trainer.fit(model)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type   | Params
-----------------------------------
0 | linear1 | Linear | 192   
1 | linear2 | Linear | 1.1 K 
2 | linear3 | Linear | 33    
3 | relu    | ReLU   | 0     
-----------------------------------
1.3 K     Trainable params
0         Non-trainable params
1.3 K     Total params
0.005     Total estimated model params size (MB)


/Users/lsacy/opt/miniconda3/envs/clcnlu/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/lsacy/opt/miniconda3/envs/clcnlu/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/lsacy/opt/miniconda3/envs/clcnlu/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1595: PossibleUserWarning: The number of training batches (26

Epoch 9: 100%|██████████| 35/35 [00:00<00:00, 144.23it/s, loss=2.02, v_num=8]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 35/35 [00:00<00:00, 139.55it/s, loss=2.02, v_num=8]


In [134]:
trainer.logged_metrics

{'train_loss': tensor(1.0916), 'val_loss': tensor(1.8803)}

In [135]:
for i, j in trainer.callback_metrics.items():
    print(i, j)

train_loss tensor(1.0916)
val_loss tensor(1.8803)


In [136]:
def optimize_model(trial):
    
    num_neurons = trial.suggest_int('num_neurons', 16, 256)
    batch_size= trial.suggest_int('batch_size', 32, 512)
    learning_rate= trial.suggest_float('learning_rate', 1e-5, 1e-1)

    model = RegressionModel(num_neurons=num_neurons, learning_rate=learning_rate, batch_size=batch_size)
    trainer = pl.Trainer(accelerator='mps')
    trainer.fit(model)
    return trainer.callback_metrics['val_loss']
    
study = optuna.create_study()
study.optimize(optimize_model, n_trials=10)

print(study.best_trial)

[I 2023-01-10 01:16:30,948] A new study created in memory with name: no-name-45b6d698-6707-466e-a7df-bebb3935c60c
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/lsacy/opt/miniconda3/envs/clcnlu/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(

  | Name    | Type   | Params
-----------------------------------
0 | linear1 | Linear | 834   
1 | linear2 | Linear | 19.5 K
2 | linear3 | Linear | 140   
3 | relu    | ReLU   | 0     
-----------------------------------
20.4 K    Trainable params
0         Non-trainable params
20.4 K    Total params
0.082     Total estimated model params size (MB)


/Users/lsacy/opt/miniconda3/envs/clcnlu/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1595: PossibleUserWarning: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 999: 100%|██████████| 7/7 [00:00<00:00, 115.49it/s, loss=2.05, v_num=17]

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████| 7/7 [00:00<00:00, 103.34it/s, loss=2.05, v_num=17]


[I 2023-01-10 01:17:41,867] Trial 0 finished with value: 1.8802931308746338 and parameters: {'num_neurons': 139, 'batch_size': 167, 'learning_rate': 0.08769622508616076}. Best is trial 0 with value: 1.8802931308746338.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type   | Params
-----------------------------------
0 | linear1 | Linear | 816   
1 | linear2 | Linear | 18.6 K
2 | linear3 | Linear | 137   
3 | relu    | ReLU   | 0     
-----------------------------------
19.6 K    Trainable params
0         Non-trainable params
19.6 K    Total params
0.078     Total estimated model params size (MB)


/Users/lsacy/opt/miniconda3/envs/clcnlu/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1595: PossibleUserWarning: The number of training batches (11) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 246:   7%|▋         | 1/15 [00:00<00:00, 119.43it/s, loss=2.03, v_num=18] 

In [46]:
# set up the logger
logger = TensorBoardLogger('lightning_logs', name='insurance')

#??? set up the early stopping callback???
early_stop_callback = EarlyStopping( monitor='val_loss', min_delta=0.00, patience=10, verbose=False, mode='min')

#??? set up the checkpoint callback???
checkpoint_callback = ModelCheckpoint(monitor='val_loss', dirpath='lightning_logs/insurance/checkpoints', filename='insurance-{epoch:02d}-{val_loss:.2f}', save_top_k=3, mode='min')

# define the model
class InsuranceModel(pl.LightningModule):
    def __init__(self, learning_rate=0.01, weight_decay=0.01):
        super().__init__()
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.linear = nn.Linear(5, 1)
        self.loss = nn.MSELoss()
        
    def forward(self, x):
        return self.linear(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        self.log('val_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        return optimizer
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X, y), batch_size=32, shuffle=True)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X, y), batch_size=32, shuffle=False)
    
    def test_dataloader(self):
        return torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X, y), batch_size=32, shuffle=False)

# define the objective function
def objective(trial):
    # sample hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-1)
    
    # instantiate the model
    model = InsuranceModel(learning_rate=learning_rate, weight_decay=weight_decay)
    
    # instantiate the trainer
    trainer = pl.Trainer(max_epochs=100, logger=logger, callbacks=[early_stop_callback, checkpoint_callback])
    
    # train the model
    trainer.fit(model)
    
    # return the validation loss
    return trainer.callback_metrics['val_loss'].item()

# instantiate the study
study = optuna.create_study(direction='minimize')

# optimize the objective function
study.optimize(objective, n_trials=100)

# print the best hyperparameters
print(study.best_params)

# print the best validation loss
print(study.best_value)

# print the best model
print(study.best_trial)

# plot the optimization history
optuna.visualization.plot_optimization_history(study)

# plot the hyperparameter relationships
optuna.visualization.plot_slice(study)

# plot the hyperparameter importances
optuna.visualization.plot_param_importances(study)

# plot the learning rate and weight decay
optuna.visualization.plot_contour(study, params=['learning_rate', 'weight_decay'])

# plot the learning rate and weight decay
optuna.visualization.plot_contour(study, params=['learning_rate', 'weight_decay'], target_name='val_loss')

# plot the learning rate and weight decay
optuna.visualization.plot_contour(study, params=['learning_rate', 'weight_decay'], target_name='train_loss')

# plot the learning rate and weight decay
optuna.visualization.plot_contour(study, params=['learning_rate', 'weight_decay'], target_name='val_loss', target_is_max=False)

# plot the learning rate and weight decay
optuna.visualization.plot_contour(study, params=['learning_rate', 'weight_decay'], target_name='train_loss', target_is_max=False)

# plot the learning rate and weight decay
optuna.visualization.plot_contour(study, params=['learning_rate', 'weight_decay'], target_name='val_loss', target_is_max=False, n_levels=20)

# plot the learning rate and weight decay
optuna.visualization.plot_contour(study, params=['learning_rate', 'weight_decay'], target_name='train_loss', target_is_max=False, n_levels=20)

# plot the learning rate and weight decay
optuna.visualization.plot_contour(study, params=['learning_rate', 'weight_decay'], target_name='val_loss', target_is_max=False, n_levels=20, sample=100)

# plot the learning rate and weight decay
optuna.visualization.plot_contour(study, params=['learning_rate', 'weight_decay'], target_name='train_loss', target_is_max=False, n_levels=20, sample=100)



[I 2023-01-08 23:11:03,308] A new study created in memory with name: no-name-efa712d4-2402-474b-bcf0-8cde9ba049bb
/var/folders/m_/_mwt902x1z595k65b7qf743m0000gn/T/ipykernel_30791/2832303483.py:52: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/var/folders/m_/_mwt902x1z595k65b7qf743m0000gn/T/ipykernel_30791/2832303483.py:53: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-1)
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, usin

/Users/lsacy/opt/miniconda3/envs/clcnlu/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/lsacy/opt/miniconda3/envs/clcnlu/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/lsacy/opt/miniconda3/envs/clcnlu/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1595: PossibleUserWarning: The number of training batches (34

Validation DataLoader 0:  35%|███▌      | 12/34 [00:00<00:00, 454.33it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch 32:  47%|████▋     | 32/68 [00:00<00:00, 447.95it/s, loss=1.01, v_num=0]

...

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name   | Type    | Params
-----------------------------------
0 | linear | Linear  | 6     
1 | loss   | MSELoss | 0     
-----------------------------------
6         Trainable params
0         Non-trainable params
6         Total params
0.000     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 68/68 [00:00<00:00, 353.75it/s, loss=0.442, v_num=0]


[I 2023-01-08 23:11:42,440] Trial 96 finished with value: 0.3060876727104187 and parameters: {'learning_rate': 0.03763460446020724, 'weight_decay': 0.0009382235989019492}. Best is trial 12 with value: 0.24191558361053467.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name   | Type    | Params
-----------------------------------
0 | linear | Linear  | 6     
1 | loss   | MSELoss | 0     
-----------------------------------
6         Trainable params
0         Non-trainable params
6         Total params
0.000     Total estimated model params size (MB)


Epoch 0:  85%|████████▌ | 58/68 [00:00<00:00, 327.97it/s, loss=0.279, v_num=0]


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

